In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import AveragePooling2D
from keras.layers import Dropout,Flatten,Dense,Input 
from keras.models import Model
from keras.optimizers import adam_v2
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

## preprocessing image data

In [3]:
directory=r'/content/sample_data/data'
categories=['with_mask','without_mask']

In [4]:
image_data=[]
labels=[]
for category in categories:
  path=os.path.join(directory,category)
  for img in os.listdir(path):
    img_path=os.path.join(path,img)
    image=load_img(img_path,target_size=(224,224))
    image = img_to_array(image)
    image=preprocess_input(image)
    
    image_data.append(image)
    labels.append(category)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# labels = to_categorical(labels)

In [22]:
image_data = np.array(image_data, dtype="float32")
labels = np.array(labels)
labels=np.reshape(labels,(labels.shape[0],))

## create model

In [25]:
(trainX, testX, trainY, testY) = train_test_split(image_data, labels,test_size=0.20, random_state=42)
datagen = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [26]:
base_conv=MobileNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3),
                     	input_tensor=Input(shape=(224, 224, 3)))

In [27]:
Adam_op=adam_v2.Adam(learning_rate=1e-4,decay=1e-4/20)

In [37]:
# my model
input=base_conv.output
maxP_1 = AveragePooling2D(pool_size=(7, 7))(input)
flatte=Flatten()(maxP_1)
dense_1=Dense(128,activation='relu')(flatte)
Drop_1=Dropout(.25)(dense_1)
dense_2=Dense(64,activation='relu')(Drop_1)
Drop_2=Dropout(.25)(dense_2)
output=Dense(1,activation='sigmoid')(Drop_2)

model=Model(base_conv.input,output)

model.compile(loss='binary_crossentropy',optimizer=Adam_op,metrics=['acc'])

In [38]:
for layer in base_conv.layers:
	layer.trainable = False

In [39]:
trainGenerator=datagen.flow(trainX, trainY, batch_size=32)
validationGenetator=datagen.flow(testX,testY,batch_size=32)

In [40]:
history=model.fit_generator(trainGenerator,steps_per_epoch=90,
                            validation_data=validationGenetator,validation_steps=50,epochs=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
90/90 [==============================] - 41s 429ms/step - loss: 0.0238 - acc: 0.9896 - val_loss: 0.0093 - val_acc: 0.9974
Epoch 2/10
90/90 [==============================] - 31s 341ms/step - loss: 0.0012 - acc: 0.9997
Epoch 3/10
90/90 [==============================] - 29s 326ms/step - loss: 6.1370e-04 - acc: 1.0000
Epoch 4/10
90/90 [==============================] - 30s 328ms/step - loss: 5.6632e-04 - acc: 1.0000
Epoch 5/10
90/90 [==============================] - 29s 326ms/step - loss: 1.5065e-04 - acc: 1.0000
Epoch 6/10
90/90 [==============================] - 29s 327ms/step - loss: 2.1436e-04 - acc: 1.0000
Epoch 7/10
90/90 [==============================] - 29s 325ms/step - loss: 7.3488e-05 - acc: 1.0000
Epoch 8/10
90/90 [==============================] - 30s 333ms/step - loss: 3.4375e-04 - acc: 1.0000
Epoch 9/10
90/90 [==============================] - 29s 326ms/step - loss: 7.5971e-05 - acc: 1.0000
Epoch 10/10
90/90 [==============================] - 29s 325ms/step - l

In [41]:
model.save('/content/sample_data/my_bestModel.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
